In [11]:
import time
import re
import json
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [12]:
# Extract url from settings.json
settings_path = Path("../settings.json")
settings = json.loads(settings_path.open().read())
url = settings["URL1"]

In [13]:
# Initialize the WebDriver
driver = webdriver.Edge()
# driver = webdriver.Chrome()
# driver.execute_script("options.addArguments("--disable-search-engine-choice-screen");").

# Open the homepage
driver.get(url)

# Wait for the page to load
time.sleep(2)

try:
    # Use the id to locate the element
    element = driver.find_element(By.ID, "cookiescript_close")
    element.click()  # Perform the click action
    print("Element clicked successfully!")

except Exception as e:
    print(f"Error occurred while finding/clicking the element: {e}")

Element clicked successfully!


In [14]:
try:
    # Use the full class name to locate the element
    element = driver.find_element(By.CLASS_NAME, "MuiTab-iconWrapper")

    # Get the text content of the element
    offers_text = element.text

    # Use regex to extract the integer, allowing for spaces as thousand separators
    offers_number = re.search(
        r"(\d+\s?)+", offers_text
    )  # This captures numbers with optional spaces
    if offers_number:
        # Remove spaces and convert to an integer
        offers_number = int(offers_number.group().replace(" ", ""))
        print("Number of offers:", offers_number)

except Exception as e:
    print(f"Error occurred while finding the element: {e}")

# Wait for results to load
time.sleep(2)

Number of offers: 10329


In [15]:
SCROLL_PAUSE_TIME = 2.0

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

offset_from_bottom = 0.2 * last_height

# Scroll down to bottom
# driver.execute_script(f"window.scrollTo(0, {last_height - offset_from_bottom});")

while True:
    # Scroll down to bottom
    driver.execute_script(f"window.scrollTo(0, {last_height - offset_from_bottom});")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [16]:
# Close the browser
driver.quit()